In [1]:
from lib import Models, Datasets, LossCalculator, Train
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


2025-02-17 09:37:00.808553: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-17 09:37:00.824042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739774220.841012  247521 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739774220.846209  247521 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 09:37:00.864699: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model, tokenizer = Models.LoadLLM("distilgpt2")
# model, tokenizer = Models.LoadLLM("meta-llama/Llama-3.2-1B")

math = Datasets.LoadMath()

print(model)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [3]:
# LossCalculator.CalcLoss(**math, max_length=1024, count=10, model=model, tokenizer=tokenizer)

In [4]:
from peft import LoraConfig, TaskType, get_peft_model

# peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj'])
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['c_attn', 'c_proj', 'c_fc', 'c_proj'])


model = get_peft_model(model, peft_config)

print(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-5): 6 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
           

/home/parshina/.local/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [5]:
Train.Trains(**math, max_length=1024, count=10000, model=model, tokenizer=tokenizer)

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

/home/parshina/workspace/AdaptorsResearch/lib/Train.py:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,7.032900
20,7.781700
30,6.313400
40,6.285600
50,5.880400
60,5.490600
70,5.456900
80,5.284900
90,5.344300
100,5.075300


KeyboardInterrupt: 

In [ ]:
# LossCalculator.CalcLoss(**math, max_length=1024, count=40, model=model, tokenizer=tokenizer)

In [6]:
save_directory = 'logs/saves'

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('logs/saves/tokenizer_config.json',
 'logs/saves/special_tokens_map.json',
 'logs/saves/vocab.json',
 'logs/saves/merges.txt',
 'logs/saves/added_tokens.json',
 'logs/saves/tokenizer.json')

In [7]:
from transformers import pipeline

generator = pipeline("text-generation", model=save_directory)

prompt = "In the near future, artificial intelligence will"

generated_text = generator(prompt, max_length=50, num_return_sequences=1)

for i, text in enumerate(generated_text):
    print(f"Generated text {i + 1}:")
    print(text['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text 1:
In the near future, artificial intelligence will take care of a wide range of needs — from power to transport — and the future of manufacturing technologies like robotics will provide valuable insights from the field. Here's where we'll examine how AI will evolve in the


In [8]:
# generator = pipeline("text-generation", model=save_directory, tokenizer=save_directory)
# generator = pipeline("text-generation", model='distilgpt2')
generator = pipeline("text-generation", model=save_directory)

prompt = "2 + 2 is"

generated_text = generator(prompt, max_length=50, num_return_sequences=10)

for i, text in enumerate(generated_text):
    print(f"Generated text {i + 1}:")
    print(text['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text 1:
2 + 2 is really nice, but it's not that useful. I'll be making a lot of changes to the game after I have finished playing and getting this little fix.
Generated text 2:
2 + 2 is a new way to find ways to change the world.


If all this is happening again in the next couple days, there will be a new way of doing magic for the worlds we live in.
You can see
Generated text 3:
2 + 2 is a 2.2% chance that it ends up killing all of the items. If it ends up killing all of the items you currently have (including any items you currently have) it will break 1% of the maximum attack.
Generated text 4:
2 + 2 is not a valid ID. You need to know your own ID.


3rd party security software will get in the way. For the first time, the first time users will be automatically warned if an account has been hacked
Generated text 5:
2 + 2 is $7.99










































Generated text 6:
2 + 2 is a perfect value, and the difference between your two is minimal. The 2+ valu